In [1]:
%cd ..

# imports:
import pyOSC3

# todo: hide/combine/simplify these imports
from domblar.sc3.client import SC3Client
from domblar.sc3.instruments import setup_instruments, assign_instrument
from domblar.players import play, play_voices
from domblar.transformations import iterate_voice_leadings
from domblar.edo import build_mos_from_edo, intervals_in_edo

/Users/gexahedron/devel/domblar


In [2]:
# setup:
# 1. open scds/vst_tracker.scd file in SuperCollider
# and run first cell
# and second cell
# and third cell

# 2. run this cell
client = SC3Client()
instruments = setup_instruments()

In [4]:
# play:

def assign_instruments():
    # assign_instrument(0, 'tub_bells', instruments, client)
    assign_instrument(0, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 37
# fixme: remove this scale, looks redundant
scale = list(range(edo))

scales = {}
scales['orgone7'] = build_mos_from_edo(7, gen=10, edo=edo, down=0, shift=0) + [edo]

cur_scale = scales['orgone7']
print(cur_scale)
print('steps -', intervals_in_edo(cur_scale, edo))
play(cur_scale, scale, edo, client, synth_idx=[0], dur=0.3)


[0, 3, 10, 13, 20, 23, 30, 37]
steps - [3, 7, 3, 7, 3, 7, 7]


In [5]:
chords = iterate_voice_leadings(
    [0, 2, 7, 11],
    [-2, 10, -2, -2],
    [0, 2, 3, 1],
    7)
print(chords)

def assign_instruments():
    for i in range(4):
        # assign_instrument(i, 'tub_bells', instruments, client)
        assign_instrument(i, 'brass/toto_africa', instruments, client)
assign_instruments()

edo = 12
scale = list(range(edo))
play(chords, scale, edo, client, synth_idx=[0, 1, 2, 3], dur=0.75)


[[0, 2, 7, 11], [-2, 5, 9, 12], [-4, 7, 10, 15], [-6, 8, 13, 17], [-8, 11, 15, 18], [-10, 13, 16, 21], [-12, 14, 19, 23]]


In [13]:
# cleanup
